In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers>=4.51.0 torch torchvision torchaudio accelerate bitsandbytes -q
!pip install sentencepiece protobuf -q
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gc
import warnings
import re
warnings.filterwarnings('ignore')
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
import torch
import gc
import re
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from typing import List, Dict, Tuple, Any
import warnings
warnings.filterwarnings('ignore')

class ArabicMedicalAnswerClassifier:
    def __init__(self):
        self.model_name = "Qwen/Qwen3-14B"
        self.tokenizer = None
        self.model = None
        self.answer_strategies = {
            '1': 'Information (answers providing information, resources, etc.)',
            '2': 'Direct Guidance (answers providing suggestions, instructions, or advice)',
            '3': 'Emotional Support (answers providing approval, reassurance, or other forms of emotional support)'
        }
        self.load_model()
    
    def load_model(self):
        """Load Qwen3-14B with optimizations for Arabic medical text classification"""
        print("Loading Qwen3-14B model for Arabic Medical Answer Classification...")
        
        # Configure quantization for memory efficiency
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            trust_remote_code=True
        )
        
        # Load model with quantization
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        
        print("Qwen3-14B model loaded successfully!")
        self.print_model_info()
    
    def print_model_info(self):
        """Print model and memory information"""
        print(f"\nModel Information:")
        print(f"Model Name: {self.model_name}")
        print(f"Model Parameters: 14.8B (13.2B non-embedding)")
        print(f"Context Length: 32,768 tokens")
        print(f"Tokenizer Vocab Size: {len(self.tokenizer):,}")
        
        if torch.cuda.is_available():
            print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
            print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    
    def classify_answer(self, answer: str, use_thinking_mode: bool = True, max_new_tokens: int = 8000) -> List[str]:
        """
        Classify Arabic medical answer into strategy categories
        
        Args:
            answer: Arabic medical answer
            use_thinking_mode: Enable Qwen3's thinking capabilities
            max_new_tokens: Maximum tokens to generate
        
        Returns:
            List[str]: extracted_strategies
        """
        
        # Create strategy descriptions in Arabic
        strategy_descriptions = """
استراتيجيات الإجابة الطبية:
(1) المعلومات - إجابات تقدم معلومات وموارد وحقائق طبية
(2) التوجيه المباشر - إجابات تقدم اقتراحات وتعليمات ونصائح محددة
(3) الدعم العاطفي - إجابات تقدم الموافقة والطمأنينة أو أشكال أخرى من الدعم العاطفي
"""
        
        messages = [
            {
                "role": "user", 
                "content": f"""أنت خبير في تصنيف الإجابات الطبية باللغة العربية. مهمتك هي تصنيف الإجابة التالية إلى استراتيجية أو أكثر من الاستراتيجيات المحددة.

{strategy_descriptions}

الإجابة المراد تصنيفها:
{answer}

تعليمات:
1. اقرأ الإجابة بعناية وحلل محتواها وأسلوبها
2. حدد الاستراتيجية أو الاستراتيجيات المناسبة (يمكن أن يكون هناك أكثر من استراتيجية واحدة)
3. اشرح سبب اختيارك لكل استراتيجية
4. في النهاية، اكتب الإجابة بالتنسيق التالي:
   "التصنيف النهائي: [1,2,3]" (استخدم الأرقام المناسبة مفصولة بفواصل)

مثال على التنسيق:
- إذا كانت الإجابة معلوماتية فقط: "التصنيف النهائي: [1]"
- إذا كانت الإجابة تحتوي على معلومات وتوجيه: "التصنيف النهائي: [1,2]"
- إذا كانت الإجابة تحتوي على المعلومات والتوجيه والدعم العاطفي: "التصنيف النهائي: [1,2,3]"
"""
            }
        ]
        
        # Apply chat template with thinking mode control
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=use_thinking_mode
        )
        
        # Tokenize input
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
        
        # Generate response
        with torch.no_grad():
            if use_thinking_mode:
                generated_ids = self.model.generate(
                    **model_inputs,
                    max_new_tokens=max_new_tokens,
                    temperature=0.6,
                    top_p=0.95,
                    top_k=20,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )
            else:
                generated_ids = self.model.generate(
                    **model_inputs,
                    max_new_tokens=max_new_tokens,
                    temperature=0.7,
                    top_p=0.8,
                    top_k=20,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )
        
        # Extract output tokens
        output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
        
        # Parse thinking content (if thinking mode is enabled)
        content = ""
        
        if use_thinking_mode:
            try:
                # Find </think> token (151668)
                index = len(output_ids) - output_ids[::-1].index(151668)
                content = self.tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip()
            except ValueError:
                content = self.tokenizer.decode(output_ids, skip_special_tokens=True).strip()
        else:
            content = self.tokenizer.decode(output_ids, skip_special_tokens=True).strip()
        
        strategies = self.extract_answer_strategies(content)
        
        return strategies
    
    def extract_answer_strategies(self, response: str) -> List[str]:
        """Extract answer strategies from Arabic response text"""
        
        # Arabic patterns for strategy extraction
        patterns = [
            r'التصنيف النهائي:\s*\[([123,\s]+)\]',  # "التصنيف النهائي: [1,2,3]"
            r'الاستراتيجيات:\s*\[([123,\s]+)\]',  # "الاستراتيجيات: [1,2,3]"
            r'التصنيف:\s*\[([123,\s]+)\]',  # "التصنيف: [1,2,3]"
            r'النتيجة:\s*\[([123,\s]+)\]',  # "النتيجة: [1,2,3]"
        ]
        
        for pattern in patterns:
            match = re.search(pattern, response, re.IGNORECASE)
            if match:
                strategies_str = match.group(1)
                strategies = [strat.strip() for strat in strategies_str.split(',')]
                return [strat for strat in strategies if strat in ['1', '2', '3']]
        
        # English patterns as fallback
        english_patterns = [
            r'Final Classification:\s*\[([123,\s]+)\]',
            r'Strategies:\s*\[([123,\s]+)\]',
            r'Classification:\s*\[([123,\s]+)\]',
        ]
        
        for pattern in english_patterns:
            match = re.search(pattern, response, re.IGNORECASE)
            if match:
                strategies_str = match.group(1)
                strategies = [strat.strip() for strat in strategies_str.split(',')]
                return [strat for strat in strategies if strat in ['1', '2', '3']]
        
        # Look for individual strategy mentions
        found_strategies = []
        for strategy in ['1', '2', '3']:
            if f'({strategy})' in response or f'[{strategy}]' in response:
                found_strategies.append(strategy)
        
        return found_strategies if found_strategies else ['1']  # Default to 'Information' if nothing found
    
    def process_test_dataset(self, df: pd.DataFrame, use_thinking: bool = True, show_progress: bool = True) -> pd.DataFrame:
        """
        Process test dataset for answer classification
        
        Args:
            df: DataFrame with 'answer' column
            use_thinking: Enable thinking mode
            show_progress: Show progress information
        
        Returns:
            DataFrame with predictions
        """
        
        print("🚀 Starting Arabic Medical Answer Classification for Test Dataset...")
        print(f"Test dataset size: {len(df)} samples")
        print("-" * 60)
        
        # Initialize result list
        predictions = []
        
        for idx, row in df.iterrows():
            if show_progress and idx % 10 == 0:
                print(f"Processing sample {idx+1}/{len(df)}")
            
            # Classify answer
            try:
                strategies = self.classify_answer(
                    row['answer'], 
                    use_thinking_mode=use_thinking
                )
                # Convert list to comma-separated string as required by submission format
                prediction_str = ', '.join(sorted(strategies))
                predictions.append(prediction_str)
            except Exception as e:
                print(f"Error processing answer {idx}: {e}")
                predictions.append('1')
            
            # Clean up memory periodically
            if idx % 20 == 0:
                self.cleanup_memory()
        
        # Create result dataframe with just the predictions
        result_df = pd.DataFrame({'prediction': predictions})
        
        return result_df
    
    def cleanup_memory(self):
        """Clean up GPU memory"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

# Initialize the classifier
print("Initializing Arabic Medical Answer Classifier with Qwen3-14B...")
classifier = ArabicMedicalAnswerClassifier()

def generate_test_predictions(test_file_path: str, output_file_path: str = 'prediction_subtask_2.tsv', use_thinking: bool = True):
    """
    Generate predictions for test dataset and save to TSV file
    
    Args:
        test_file_path: Path to test dataset TSV file
        output_file_path: Path for output predictions file
        use_thinking: Enable thinking mode
    """
    
    print("🚀 Loading test dataset...")
    
    # Load test dataset
    try:
        df = pd.read_csv(test_file_path, sep='\t',header=None)
        df.columns=['answer']
        # df=df[:1]
        print(f"✅ Test dataset loaded successfully: {len(df)} samples")
    except Exception as e:
        print(f"❌ Error loading test dataset: {e}")
        return
    
    # Check if 'answer' column exists
    if 'answer' not in df.columns:
        print("❌ Error: 'answer' column not found in test dataset")
        print(f"Available columns: {list(df.columns)}")
        return
    
    # Process test dataset
    print("🔍 Processing test dataset...")
    results_df = classifier.process_test_dataset(df, use_thinking=use_thinking)
    
    # Save predictions to TSV file
    try:
        # Save as TSV without header, just predictions
        results_df['prediction'].to_csv(output_file_path, sep='\t', index=False, header=False)
        print(f"✅ Predictions saved to: {output_file_path}")
        print(f"📄 Total predictions: {len(results_df)}")
        
        # Show sample predictions
        print("\n📋 Sample predictions:")
        for i in range(min(5, len(results_df))):
            print(f"Sample {i+1}: {results_df['prediction'].iloc[i]}")
            
    except Exception as e:
        print(f"❌ Error saving predictions: {e}")
    
    # Clean up memory
    classifier.cleanup_memory()
    
    return results_df

def get_model_status():
    """Display current model status"""
    print("\n📊 Model Status:")
    print(f"Model: {classifier.model_name}")
    print(f"Model loaded: {'✅' if classifier.model else '❌'}")
    if torch.cuda.is_available():
        print(f"GPU memory usage: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    classifier.print_model_info()

# Main execution
if __name__ == "__main__":
    print("🚀 Arabic Medical Answer Classifier for Test Dataset Ready!")
    
    # Replace with your actual test dataset path
    TEST_DATASET_PATH = '/kaggle/input/testdf/subtask2_input_test.tsv'  # Update this path
    
    # Generate predictions
    # Uncomment the following line and update the path:
    generate_test_predictions(TEST_DATASET_PATH)
    
    # For demonstration with sample data (remove this in actual usage):
    # print("📚 Creating sample test to demonstrate functionality...")
    # sample_data = pd.DataFrame({
    #     'answer': [
    #         'الصداع المستمر قد يكون علامة على حالات مختلفة. من المهم استشارة طبيب مختص لتقييم الحالة بشكل صحيح.',
    #         'يمكنك تناول مسكن للألم واستشارة الطبيب إذا استمر الألم. لا تقلق، معظم الحالات قابلة للعلاج.',
    #         'السكري مرض مزمن يؤثر على مستويات السكر في الدم. يتطلب متابعة طبية منتظمة وإدارة النظام الغذائي.'
    #     ]
    # })
    
    # print("Processing sample data...")
    # sample_results = classifier.process_test_dataset(sample_data, use_thinking=True)
    # print("\nSample predictions:")
    # for i, pred in enumerate(sample_results['prediction']):
    #     print(f"Answer {i+1}: {pred}")
    
    # # Save sample predictions
    # sample_results['prediction'].to_csv('sample_prediction_subtask_2.tsv', sep='\t', index=False, header=False)
    # print("Sample predictions saved to: sample_prediction_subtask_2.tsv")